## Load packages

In [1]:
using DataFrames, StatsPlots, CSV, HTTP
using Distributions, Turing, MCMCChains

## Load Data

In [2]:
data = "https://raw.githubusercontent.com/avehtari/ROS-Examples/master/Incentives/data/incentives.csv"
incentives  = CSV.File(HTTP.get(data).body, delim=",") |> DataFrame
first(incentives , 6)

,rr_diff,value,prepay,gift,burden
,Int64,Float64,Int64,Int64,Int64
1,3,1.24151,1,0,0
2,6,2.46624,1,1,0
3,9,14.7135,0,0,1
4,4,24.6288,0,0,1
5,6,43.1172,0,0,1
6,13,17.314,0,0,1


In [3]:
model_data = Matrix(select(incentives,Not(:rr_diff)))
targets = incentives.rr_diff;

## Normal linear regression

In [4]:
@model function m1(x, y)
    # Set variance prior.
    σ ~ truncated(Normal(0, 100), 0, Inf)
    
    # Set intercept prior.
    intercept ~ Normal(0,50)
    
    # Set the priors on our coefficients.
    nfeatures = size(x, 2)
    coefficients ~ MvNormal(nfeatures, 10)
    
    # Calculate all the mu terms.
    mu = intercept .+ x * coefficients
    y ~ MvNormal(mu, σ)
end;

In [5]:
model = m1(model_data, targets)
chain = sample(model, NUTS(), 1_000);

┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC C:\Users\tburch\.julia\packages\AdvancedHMC\MIxdK\src\hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC C:\Users\tburch\.julia\packages\AdvancedHMC\MIxdK\src\hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC C:\Users\tburch\.julia\packages\AdvancedHMC\MIxdK\src\hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC C:\Users\tburch\.julia\packages\AdvancedHMC\MIxdK\src\hamiltonian.jl:47
┌ Info: Found initial step size
│   ϵ = 0.0015625
└ @ Turing.Inference C:\Users\tburch\.julia\packages\T

In [6]:
chain

Chains MCMC chain (1000×18×1 Array{Float64, 3}):

Iterations        = 1:1000
Thinning interval = 1
Chains            = 1
Samples per chain = 1000
parameters        = coefficients[1], coefficients[2], coefficients[3], coefficients[4], intercept, σ
internals         = acceptance_rate, hamiltonian_energy, hamiltonian_energy_error, is_accept, log_density, lp, max_hamiltonian_energy_error, n_steps, nom_step_size, numerical_error, step_size, tree_depth

Summary Statistics
       parameters      mean       std   naive_se      mcse         ess      rh ⋯
           Symbol   Float64   Float64    Float64   Float64     Float64   Float ⋯

  coefficients[1]    0.1224    0.0444     0.0014    0.0015    643.6332    0.99 ⋯
  coefficients[2]    3.7236    1.9669     0.0622    0.0693    722.1928    1.00 ⋯
  coefficients[3]   -5.0501    2.0964     0.0663    0.0624    806.5378    0.99 ⋯
  coefficients[4]    2.8697    1.5347     0.0485    0.0349    954.6031    1.00 ⋯
        intercept    1.6898    1.6921     